# SIDIA Challenge Amazonas do Espaço 🌏🌲🌳
## Autor: Douglas Queiroz G B.


### Objetivo do Challenge

O desafio apresentado é criar um modelo que rotule essas imagens com base nas condições atmosféricas e no uso do solo, com o objetivo geral de rastrear a pegada de carbono humana na maior floresta tropical do mundo. Vodê pode encontrar a competição original [aqui](http://www.kaggle.com/c/planet-understanding-the-amazon-from-space).

### Entendendo o Dataset

A cada minuto, o mundo perde uma área de floresta do tamanho de 48 campos de futebol. E o desmatamento na Bacia Amazônica é responsável pela maior parte, contribuindo para a redução da biodiversidade, perda de habitat, mudança climática e outros efeitos devastadores. Porém, dados melhores sobre a localização do desmatamento e invasão humana nas florestas podem ajudar os governos e as partes interessadas locais a responder com mais rapidez e eficácia.

Os chips para esta competição foram derivados dos produtos de cena analítica full-frame da Planet usando nossos satélites de 4 bandas em órbita sincronizada com o sol (SSO) e a órbita da Estação Espacial Internacional (ISS).

Os rótulos podem ser amplamente divididos em três grupos: **_condições atmosféricas, fenômenos comuns de uso do solo / cobertura do solo e fenômenos raros de uso do solo / cobertura do solo_**. Cada chip terá um e potencialmente mais de um rótulo atmosférico e zero ou mais rótulos comuns e raros. **Chips rotulados como turvos não devem ter outros rótulos, mas pode haver erros de rotulagem.**

As nuvens são um grande desafio para imagens passivas de satélite, e a cobertura de nuvens e pancadas de chuva diárias na bacia amazônica podem complicar significativamente o monitoramento na área. Por esse motivo, optamos por incluir uma etiqueta de cobertura de nuvens para cada chip. Esses rótulos refletem de perto o que se veria em uma previsão do tempo local: claro, parcialmente nublado, nublado e neblina.

Nota lateral: Os rótulos comuns neste conjunto de dados são floresta tropical, agricultura, rios, vilas/cidades e estradas.

### Estrutura Dataset

Quadrados de imagens de alta resolução (256 x 256) em quatro bandas (RGB + IR) do Planet Flock 2 Satellites. Cada bloco pode ter vários rótulos (comuns e menos comuns), mas apenas um dos rótulos de cobertura de nuvem.

Rótulos comuns | Rótulos menos comuns | Cloud Cover Labels
------------ | ------------- | -------------
Primary Rain Forest | Slash and Burn | Clear
Water (Rivers & Lakes) | Selective Logging | Partly cloudy
Habitation | Blooming | Cloudy
Agriculture | Conventional Mining | Haze
Road | Artisinal Mining |
Cultivation | Blow Down |
Bare Ground | |

### File formats

Este é um conjunto de dados de competição que foi contribuído pela Planet. O conjunto de dados contém arquivos csv de treinamento e imagens de treinamento / teste de chips de imagem de satélite da floresta amazônica.

- **rain.csv** - a list of training file names and their labels, the labels are space-delimited
- **sample_submission.csv** - correct format of submission, contains all the files in the test set. For more information about the submission file, please go to the Evaluation page.
- **[train/test]-tif-v2.tar.7z** - tif files for the training/test set (updated: May 5th, 2017)
- **[train/test]-jpg[-additional].tar.7z** - jpg files for the trainin/test set (updated: May 5th, 2017)
- **Kaggle-planet-[train/test]-tif.torrent** - a BitTorrent file for downloading [train/test]-tif-v2.tar.7z 



## Arquiteturas Implementadas

### VGG-16 

A rede VGG é uma rede neural convolucional inventada por [Simonyan e Zisserman do Visual Geometry Group (VGG)](https://www.robots.ox.ac.uk/~vgg/research/very_deep/) da University of Oxford em 2014. Tornou-se mais versada na comunidade de visão computacional após ser nomeada vice-campeã da tarefa de classificação ILSVRC de 2014 . É frequentemente associado ao VGG-19, com a diferença de que o VGG-16 tem 16 camadas com pesos treináveis em vez de 19, daí o nome. 

In [ ]:
# Executando jupyter com acesso root caso vc este encontrando dificuldade em instalar as dependências
# sudo -E env "PATH=$PATH" jupyter notebook --allow-root

### Importando e baixando bibliotecas necessáris

In [ ]:
import os
import sys
import pathlib
import numpy as np 
import pandas as pd 
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras

from keras import models
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.utils import shuffle
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split 

import cv2
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

### Baixando e extraindo Dataset

O dataset já existe no kaggle

# Pré Procesamento

### Data Visualization + Exploration

In [ ]:
train_df = pd.read_csv("/kaggle/input/planets-dataset/planet/planet/train_classes.csv")
test_df = pd.read_csv("/kaggle/input/planets-dataset/planet/planet/sample_submission.csv")


In [ ]:
train_classes = train_df[:]['tags']

no_classes = len(train_classes.unique())
print(f'Given {len(train_classes)} samples, there are {no_classes} unique classes.', '\n')

train_df.head()

In [ ]:
# Normalizando nomes das tags
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in train_df['tags'].values])))

# Criando uma coleção key, value para categorizar de forma numerica nossos labels
label_map = {l: i for i, l in enumerate(labels)}
print(f'labels = {labels},\n length = {len(labels)}', '\n')

print(f'label_map = {label_map},\n length = {len(label_map)}')

A maioria das imagens tem dois rótulos, três e quatro rótulos são bastante iguais em número e um, cinco, e seis rótulos não aparecem com muita frequência. Imagens que podem ser classificadas sob sete, oito ou nove rótulos raramente aparecem no conjunto de dados. Interessante; há um grande desequilíbrio aqui.

Seguindo em frente, é uma boa ideia visualizar algumas das imagens para obter uma visão de como esses rótulos devem ser:

In [ ]:

new_style = {'grid': False}
plt.rc('axes', **new_style)
_, ax = plt.subplots(3, 3, sharex='col', sharey='row', figsize=(15, 15))
i = 0
for f, tags in train_df[:9].values:
    img = cv2.imread('/kaggle/input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(f))
    ax[i // 3, i % 3].imshow(img)
    ax[i // 3, i % 3].set_title('{} - {}'.format(f, tags))
  
    i += 1
    
plt.show()

In [ ]:
# Load the train-jpg file path

train_img_dir = pathlib.Path('/kaggle/input/planets-dataset/planet/planet/train-jpg')
test_img_dir = pathlib.Path('/kaggle/input/planets-dataset/planet/planet/test-jpg')
test_add_img_dir = pathlib.Path('/kaggle/input/planets-dataset/test-jpg-additional/test-jpg-additional')

train_img_path = sorted(list(train_img_dir.glob('*.jpg')))

train_img_count = len(train_img_path)
print('Quantidade de imgs chips para treino: ',str(train_img_count))

In [ ]:
# first test jpg file path

test_img_path = sorted(list(test_img_dir.glob('*.jpg')))

test_img_count = len(test_img_path)
print('Quantidade de imgs chips para testes: ',str(test_img_count))

In [ ]:
# second test jpg file path

test_add_img_path = sorted(list(test_add_img_dir.glob('*.jpg')))

test_add_img_count = len(test_add_img_path)
print('Quantidade de imgs chips para testes adicional: ',str(test_add_img_count))

In [ ]:
# verifica se o número de imagens jpg seja igual ao número de amostras no arquivo csv para cada conjunto de dados

# train
if len(train_img_path) == len(train_df):
    print('Dataset de treino com a mesma quantidade de samples listada no csv')
#caso não seja igual a execução para aqui
assert len(train_img_path) == len(train_df) 


# test
if len(test_img_path)+len(test_add_img_path) == len(test_df):
    print('Dataset de testes com a mesma quantidade de samples listada no csv')
#caso não seja igual a execução para aqui
assert len(test_img_path)+len(test_add_img_path) == len(test_df)

### Data Preprocessing

In [ ]:
input_size = 64
input_channels = 3

batch_size = 64

In [ ]:
x_train = []
y_train = []

for f, tags in tqdm(train_df.values, miniters=1000):
    img = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(f))
    img = cv2.resize(img, (input_size, input_size))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    x_train.append(img)
    y_train.append(targets)
        
x_train = np.array(x_train, np.float32)
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

In [ ]:
x_test = []

test_image_names = os.listdir(test_img_dir)

n_test = len(test_image_names)
test_classes = test_df.iloc[:n_test, :]
add_classes = test_df.iloc[n_test:, :]

test_add_image_names = os.listdir(test_add_img_dir)

for img_name, _ in tqdm(test_classes.values, miniters=1000):
    img = cv2.imread(str(test_img_dir) + '/{}.jpg'.format(img_name))
    x_test.append(cv2.resize(img, (64, 64)))
    
for img_name, _ in tqdm(add_classes.values, miniters=1000):
    img = cv2.imread(str(test_add_img_dir) + '/{}.jpg'.format(img_name))
    x_test.append(cv2.resize(img, (64, 64)))

x_test = np.array(x_test, np.float32)
print(x_test.shape)

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size=0.2)

# Construindo o modelo

In [ ]:
base_model = VGG16(include_top=False,
                   weights='imagenet',
                   input_shape=(input_size, input_size, input_channels))

model = Sequential()
model.add(BatchNormalization(input_shape=(input_size, input_size, input_channels)))

model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(17, activation='sigmoid'))

In [ ]:

model.compile(loss='binary_crossentropy',optimizer="SGD", metrics=['accuracy'])
    
callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0),
                ModelCheckpoint(filepath='weights/best_weights',
                                 save_best_only=True,
                                 save_weights_only=True)]
model.summary()

In [ ]:
history = model.fit(x=X_train, y=Y_train, validation_data=(X_valid, Y_valid),
                  batch_size=batch_size,verbose=2, epochs=15,callbacks=callbacks,shuffle=True)

In [ ]:
model.save('./models/vgg16-amazon2')

In [ ]:
model_back = models.load_model("./models/vgg16-amazon2")

In [ ]:
p_valid = model_back.predict(X_valid, batch_size = batch_size, verbose=1)

print('Acurácia: ',fbeta_score(Y_valid, np.array(p_valid) > 0.18, beta=2, average='samples'))

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
y_pred = []
p_test = model_back.predict(x_test, batch_size=batch_size, verbose=2)
y_pred.append(p_test)

In [ ]:
result = np.array(y_pred[0])
for i in range(1, len(y_pred)):
    result += np.array(y_pred[i])
result = pd.DataFrame(result, columns=labels)

In [ ]:
# Translating the probability predictions to the unique labels
preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.loc[[i]]
    a = a.apply(lambda x: x>0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

In [ ]:
# Replacing the tags columns with the predicted labels
test_df['tags'] = preds
test_df.head()

In [ ]:
# Converting the dataframe to a csv file for submission
test_df.to_csv('sample_testes_results.csv', index=False)

# Conclusão

O modelo proposto obteve uma acurácia de 91%, o mesmo conseguiu rotular de forma satisfatória os chips do dataset de testes. Foi observado que iterações acima de 15 passos ocasionava overfit e começava a convergir. Vale ressaltar que a acurácia obtida foi relativamente boa, levando em consideração que foi utilizado chips/imagens JPG com baixa qualidade, não foi possível utilizar os datastes com imagens TIF com maior qualidade, pois o Hardware usado não suportou todo o processamento. 